# Criar VENV

https://oandersonbm.medium.com/usando-python-no-vs-code-com-ambiente-virtual-venv-ecef7959b652

In [1]:
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
from datetime import datetime
import locale

# Definindo a configuração local para o Brasil (pt_BR)
locale.setlocale(locale.LC_ALL, 'pt_BR')

# Personalizando para ver todas as colunas do DataFrame
pd.set_option('display.max_columns', None)

# Base de dados

In [2]:
# Realizando a junção dos 3 arquivos .CSV num único DataFrame

# Caminhos para os arquivos .CSV
caminho_1 = r"C:\Users\User\OneDrive\Trabalho\ESA 2023\Dashboard_ESA\base_dados\Cel Manfrini.csv"

caminho_2 = r"C:\Users\User\OneDrive\Trabalho\ESA 2023\Dashboard_ESA\base_dados\Cel Manfrini(1).csv"

caminho_3 = r"C:\Users\User\OneDrive\Trabalho\ESA 2023\Dashboard_ESA\base_dados\Cel Manfrini(2).csv"

caminho_4 = r"C:\Users\User\OneDrive\Trabalho\ESA 2023\Dashboard_ESA\base_dados\Cel Manfrini(3).csv"


# Leitura dos arquivos .CSV em DataFrames individuais
df1 = pd.read_csv(caminho_1)
df2 = pd.read_csv(caminho_2)
df3 = pd.read_csv(caminho_3)
df4 = pd.read_csv(caminho_4)

# Concatenação dos DataFrames em um único DataFrame
df = pd.concat([df1, df2, df3, df4], ignore_index=True)

# Excluindo linhas que contenham a string "Tela" em qualquer coluna
df = df[~df.apply(lambda row: row.astype(str).str.contains('Tela')).any(axis=1)]

# Transformando o tipo de dado das colunas de valores monetários
df['A_LIQUIDAR'] = df['A_LIQUIDAR'].apply(lambda x: float(x.replace('.', '').replace(',', '.')))
df['LIQUIDADO_A_PAGAR'] = df['LIQUIDADO_A_PAGAR'].apply(lambda x: float(x.replace('.', '').replace(',', '.')))
df['TOTAL_A_PAGAR'] = df['TOTAL_A_PAGAR'].apply(lambda x: float(x.replace('.', '').replace(',', '.')))
df['PAGO'] = df['PAGO'].apply(lambda x: float(x.replace('.', '').replace(',', '.')))

# Convertendo a coluna DATA para datetime
df['DATA'] = pd.to_datetime(df['DATA'], format='%d/%m/%y')

# Convertendo a coluna DIAS em número inteiro
df['DIAS'] = df['DIAS'].astype(int)



# Trabalhando com um DATAFRAME voltado para controle dos SALDOS

In [3]:
df_saldos = df[['SIGLA UG', 'ANO', 'NOME_CREDOR', 'DATA', 'DIAS', 'ND','NDSI', 'NOME_NDSI', 'NE', 'PI', 'NOME_PI','TIPO', 'NOME_UGR', 'A_LIQUIDAR', 'LIQUIDADO_A_PAGAR', 'TOTAL_A_PAGAR', 'PAGO']]

In [4]:
# Saldos Totais
total_pago = df['PAGO'].sum().round(2)
total_a_liquidar = df['A_LIQUIDAR'].sum().round(2)
total_liquidado_a_pagar = df['LIQUIDADO_A_PAGAR'].sum().round(2)
total_a_pagar = df['TOTAL_A_PAGAR'].sum().round(2)


print(f'O total pago pela ESA até o momento foi: {locale.currency(total_pago, grouping=True)}')
print(f'O total de saldo a liquidar pela ESA até o momento foi: {locale.currency(total_a_liquidar, grouping=True)}')
print(f'O total de saldo liquidado a pagar pela ESA até o momento foi: {locale.currency(total_liquidado_a_pagar, grouping=True)}')
print(f'O total de saldo a pagar pela ESA até o momento foi: {locale.currency(total_a_pagar, grouping=True)}')

O total pago pela ESA até o momento foi: R$ 27.640.427,02
O total de saldo a liquidar pela ESA até o momento foi: R$ 5.501.509,04
O total de saldo liquidado a pagar pela ESA até o momento foi: R$ 119.130,13
O total de saldo a pagar pela ESA até o momento foi: R$ 5.620.639,17


In [5]:
# Saldo a liquidar por ND
cols = ['ND', 'A_LIQUIDAR']

df_aux = df.loc[:, cols].groupby('ND').sum().sort_values(by= 'A_LIQUIDAR', ascending=False).reset_index()
df_aux = df_aux[df_aux['A_LIQUIDAR'] > 0]
fig = px.bar(df_aux, 
             x='ND', 
             y='A_LIQUIDAR',
             title='Saldo a liquidar por ND',
             labels={'ND':'Natureza da Despesa', 'A_LIQUIDAR':'Saldo a liquidar'},
             text_auto='.3s',
             height=500,
             log_y=True,
             color='A_LIQUIDAR',
             color_continuous_scale = 'reds',
             template='plotly_dark')
fig.update_traces(textposition = 'outside', cliponaxis = False, hovertemplate=None)
fig.update_layout(hovermode="x unified")

fig.show()

In [6]:
# Saldo a liquidar por Subitem
cols = ['NDSI', 'A_LIQUIDAR']

df_aux = df.loc[:, cols].groupby('NDSI').sum().sort_values(by= 'A_LIQUIDAR', ascending=False).reset_index()
df_aux = df_aux[df_aux['A_LIQUIDAR'] > 0]
fig = px.bar(df_aux, 
             x='NDSI', 
             y='A_LIQUIDAR',
             title='Saldo a liquidar por NDSI',
             labels={'NDSI':'Subitem da despesa', 'A_LIQUIDAR':'Saldo a liquidar'},
             text_auto='.3s',
             height=600,
             log_y=True,
             color='A_LIQUIDAR',
             color_continuous_scale = 'reds',
             template='plotly_dark')
fig.update_traces(textposition = 'outside', cliponaxis = False, hovertemplate=None)
fig.update_layout(hovermode="x unified")

fig.show()

In [7]:
# Saldo a liquidar por tipo de PI
cols = ['NOME_PI', 'A_LIQUIDAR']

df_aux = df.loc[:, cols].groupby('NOME_PI').sum().sort_values(by= 'A_LIQUIDAR', ascending=False).reset_index()
df_aux = df_aux[df_aux['A_LIQUIDAR'] > 0]
fig = px.bar(df_aux, 
             x='NOME_PI', 
             y='A_LIQUIDAR',
             title='Saldo a liquidar por PI',
             labels={'NOME_PI':'Tipo de PI', 'A_LIQUIDAR':'Saldo a liquidar'},
             text_auto='.3s',
             height=800,
             log_y=True,
             color='A_LIQUIDAR',
             color_continuous_scale = 'reds',
             template='plotly_dark')
fig.update_traces(textposition = 'outside', cliponaxis = False, hovertemplate=None)
fig.update_layout(hovermode="x unified")

fig.show()

In [8]:
# Saldo a liquidar por tipo de Empenho
cols = ['TIPO', 'A_LIQUIDAR']

df_aux = df.loc[:, cols].groupby('TIPO').sum().sort_values(by= 'A_LIQUIDAR', ascending=False).reset_index()
df_aux = df_aux[df_aux['A_LIQUIDAR'] > 0]
fig = px.bar(df_aux, 
             x='TIPO', 
             y='A_LIQUIDAR',
             title='Saldo a liquidar por tipo de empenho',
             labels={'TIPO':'Tipo de Empenho', 'A_LIQUIDAR':'Saldo a liquidar'},
             text_auto='.3s',
             height=400,
             log_y=True,
             color='A_LIQUIDAR',
             color_continuous_scale = 'reds',
             template='plotly_dark')
fig.update_traces(textposition = 'outside', cliponaxis = False, hovertemplate=None)
fig.update_layout(hovermode="x unified")

fig.show()

In [9]:
# Saldo a liquidar por ano
cols = ['ANO', 'A_LIQUIDAR']

df_aux = df.loc[:, cols].groupby('ANO').sum().sort_values(by= 'A_LIQUIDAR', ascending=False).reset_index()
df_aux = df_aux[df_aux['A_LIQUIDAR'] > 0]
fig = px.bar(df_aux, 
             x='ANO', 
             y='A_LIQUIDAR',
             title='Saldo a liquidar por ano do empenho',
             labels={'ANO':'Ano do Empenho', 'A_LIQUIDAR':'Saldo a liquidar'},
             text_auto='.3s',
             height=400,
             log_y=True,
             color='A_LIQUIDAR',
             color_continuous_scale = 'reds',
             template='plotly_dark')
fig.update_traces(textposition = 'outside', cliponaxis = False, hovertemplate=None)
fig.update_layout(hovermode="x unified")

fig.show()

In [10]:
# Saldo a liquidar por tipo de NE
cols = ['TIPO_NE', 'A_LIQUIDAR']

df_aux = df.loc[:, cols].groupby('TIPO_NE').sum().sort_values(by= 'A_LIQUIDAR', ascending=False).reset_index()
df_aux = df_aux[df_aux['A_LIQUIDAR'] > 0]
fig = px.bar(df_aux, 
             x='TIPO_NE', 
             y='A_LIQUIDAR',
             title='Saldo a liquidar por ano do empenho',
             labels={'TIPO_NE':'Tipo de Nota de Empenho', 'A_LIQUIDAR':'Saldo a liquidar'},
             text_auto='.3s',
             height=400,
             log_y=True,
             color='A_LIQUIDAR',
             color_continuous_scale = 'reds',
             template='plotly_dark')
fig.update_traces(textposition = 'outside', cliponaxis = False, hovertemplate=None)
fig.update_layout(hovermode="x unified")

fig.show()

In [11]:
# Saldo a liquidar por tipo de NE
cols = ['NOME_UGR', 'A_LIQUIDAR']

df_aux = df.loc[:, cols].groupby('NOME_UGR').sum().sort_values(by= 'A_LIQUIDAR', ascending=False).reset_index()
df_aux = df_aux[df_aux['A_LIQUIDAR'] > 0]
fig = px.bar(df_aux, 
             x='NOME_UGR', 
             y='A_LIQUIDAR',
             title='Saldo a liquidar por ano do empenho',
             labels={'NOME_UGR':'Nome das UGR', 'A_LIQUIDAR':'Saldo a liquidar'},
             text_auto='.3s',
             height=400,
             log_y=True,
             color='A_LIQUIDAR',
             color_continuous_scale = 'reds',
             template='plotly_dark')
fig.update_traces(textposition = 'outside', cliponaxis = False, hovertemplate=None)
fig.update_layout(hovermode="x unified")

fig.show()

In [12]:
# Saldo a liquidar por ND
cols = ['ND', 'A_LIQUIDAR', 'LIQUIDADO_A_PAGAR', 'PAGO']

df_aux = df.loc[:, cols].groupby('ND').sum().sort_values(by= 'A_LIQUIDAR', ascending=False).reset_index()
df_aux = df_aux[df_aux['A_LIQUIDAR'] > 0]
# Criando o gráfico
fig = go.Figure()

# Adicionando as colunas y1, y2 e y3 no eixo y
fig.add_trace(go.Bar(x=df_aux['ND'], y=df_aux['A_LIQUIDAR'], name='A Liquidar'))
fig.add_trace(go.Bar(x=df_aux['ND'], y=df_aux['LIQUIDADO_A_PAGAR'], name='Liquidado a pagar'))
fig.add_trace(go.Bar(x=df_aux['ND'], y=df_aux['PAGO'], name='Pago'))

fig.update_traces(textposition = 'outside', cliponaxis = False, hovertemplate=None)
fig.update_layout(hovermode="x unified")
fig.update_yaxes(type="log")

fig.show()

# Suporte para o Dashboard

In [13]:
cols = ['NE', 'A_LIQUIDAR', 'DIAS', 'NOME_NDSI', 'NDSI']

df_aux = df.loc[:, cols].sort_values(by='DIAS', ascending=False)
df_aux = df_aux[df_aux['A_LIQUIDAR'] != 0].head(10)
df_aux

,NE,A_LIQUIDAR,DIAS,NOME_NDSI,NDSI
1152,2022NE001087,2397.00,406,MANUTENCAO E CONSERVACAO DE EQUIPAMENTOS DE TIC,33904012
1186,2022NE001086,3466.70,406,MANUTENCAO E CONSERVACAO DE EQUIPAMENTOS DE TIC,33904012
1185,2022NE001085,5543.35,406,MANUT. E CONSERV. DE MAQUINAS E EQUIPAMENTOS,33903917
1175,2022NE002421,5758.75,351,SERVICOS GRAFICOS E EDITORIAIS,33903963
1438,2022NE001843,1016.25,351,SERVICOS GRAFICOS E EDITORIAIS,33903963
965,2022NE002482,155796.97,337,MANUTENCAO E CONSERV. DE BENS IMOVEIS,33903916
966,2022NE002483,235144.91,337,MANUTENCAO E CONSERV. DE BENS IMOVEIS,33903916
967,2022NE002701,19358.27,323,MANUTENCAO E CONSERV. DE BENS IMOVEIS,33903916
1168,2022NE002702,12733.76,323,MANUTENCAO E CONSERV. DE BENS IMOVEIS,33903916
301,2023NE000001,108008.87,310,SERVICOS DE ENERGIA ELETRICA,33903943


In [14]:
# Saldo a liquidar por tipo de NE
cols = ['NE', 'A_LIQUIDAR', 'DIAS', 'NOME_NDSI', 'NDSI']
df_aux = df.loc[:, cols].sort_values(by='DIAS', ascending=False)
df_aux = df_aux[df_aux['A_LIQUIDAR'] != 0].head(10)
fig = px.bar(df_aux, 
             x='NE', 
             y='A_LIQUIDAR',
             title='10 empenhos com mais tempo na tela',
             labels={'NE':'Nota de Empenho', 'A_LIQUIDAR':'Saldo a liquidar'},
             text_auto='.3s',
             color='DIAS',
             color_continuous_scale = 'reds',
             log_y=True,
             template='plotly_dark',
             hover_name="NE", hover_data={'DIAS': True, 'A_LIQUIDAR': True, 'NOME_NDSI': True, 'NDSI': True})
fig.update_traces(textposition = 'outside')


fig.show()

## Informações para SideBar

In [15]:
# Qtde de empenhos realizados até o momento
df.shape[0]

3335

# Requisição em API para consulta a CNPJ

In [16]:
import requests
from pprint import pprint

url = "https://receitaws.com.br/v1/cnpj/21331404000138"

headers = {"Accept": "application/json"}

response = requests.get(url, headers=headers)

pprint(response.json())

{'abertura': '03/11/2014',
 'atividade_principal': [{'code': '79.11-2-00', 'text': 'Agências de viagens'}],
 'atividades_secundarias': [{'code': '59.11-1-99',
                             'text': 'Atividades de produção cinematográfica, '
                                     'de vídeos e de programas de televisão '
                                     'não especificadas anteriormente'},
                            {'code': '63.99-2-00',
                             'text': 'Outras atividades de prestação de '
                                     'serviços de informação não especificadas '
                                     'anteriormente'},
                            {'code': '79.12-1-00',
                             'text': 'Operadores turísticos'},
                            {'code': '79.90-2-00',
                             'text': 'Serviços de reservas e outros serviços '
                                     'de turismo não especificados '
                                    

In [17]:
def consulta_cnpj(numero_cnpj):
    url = f"https://receitaws.com.br/v1/cnpj/{numero_cnpj}"

    headers = {"Accept": "application/json"}

    response = requests.get(url, headers=headers)
    
    dic_fornecedor = response.json()
    
    return dic_fornecedor

# Criar um dicionário vazio fora da função
dicionario = {}

# Chamar a função e atribuir o resultado ao dicionário
dicionario = consulta_cnpj('81243735001977')

# Imprimir o dicionário
pprint(dicionario)

{'abertura': '28/08/2015',
 'atividade_principal': [{'code': '26.21-3-00',
                          'text': 'Fabricação de equipamentos de informática'}],
 'atividades_secundarias': [{'code': '26.22-1-00',
                             'text': 'Fabricação de periféricos para '
                                     'equipamentos de informática'},
                            {'code': '27.59-7-99',
                             'text': 'Fabricação de outros aparelhos '
                                     'eletrodomésticos não especificados '
                                     'anteriormente, peças e acessórios'},
                            {'code': '46.19-2-00',
                             'text': 'Representantes comerciais e agentes do '
                                     'comércio de mercadorias em geral não '
                                     'especializado'},
                            {'code': '46.51-6-01',
                             'text': 'Comércio atacadista de equipam

In [18]:
pprint(dicionario)

{'abertura': '28/08/2015',
 'atividade_principal': [{'code': '26.21-3-00',
                          'text': 'Fabricação de equipamentos de informática'}],
 'atividades_secundarias': [{'code': '26.22-1-00',
                             'text': 'Fabricação de periféricos para '
                                     'equipamentos de informática'},
                            {'code': '27.59-7-99',
                             'text': 'Fabricação de outros aparelhos '
                                     'eletrodomésticos não especificados '
                                     'anteriormente, peças e acessórios'},
                            {'code': '46.19-2-00',
                             'text': 'Representantes comerciais e agentes do '
                                     'comércio de mercadorias em geral não '
                                     'especializado'},
                            {'code': '46.51-6-01',
                             'text': 'Comércio atacadista de equipam

In [19]:
dicionario['nome']

'POSITIVO TECNOLOGIA S.A.'

In [20]:
dicionario['telefone']

'(41) 3316-7700'

In [21]:
dicionario['email']

'fiscal@positivo.com.br'

In [22]:
dicionario['logradouro'] + ', ' + dicionario['numero'] + ' - ' + dicionario['bairro'] + ' - ' + dicionario['municipio'] + ' / ' + dicionario['uf']

'R JAVARI, 1255 - DISTRITO INDUSTRIAL I - MANAUS / AM'

In [23]:
# filtrando dos dados da coluna FAV
df_cnpj = df[df['NOME_PI'] != 'PROFISSIONAL DE SAUDE AUTONOMO']
df_cnpj = df[df['NOME_PI'] != 'ATENDIMENTO MEDICO-HOPITALR/FC']
df_cnpj = df[df['NOME_PI'] != 'IND-INDENIZACOES']
df_cnpj = df[df['NOME_PI'] != 'IND - INDENIZACOES']
df_cnpj = df[df['NOME_PI'] != 'INDENIZACAO DESPESAS.']
df_cnpj = df[df['NOME_PI'] != 'AUXILIO FINANCEIRO NÃO INDENIZÁVEL']
df_cnpj = df[df['NOME_PI'] != 'INRE - RESTITUICAO E RESSARCIMENTO']
df_cnpj = df[df['NOME_NDSI'] != 'OUTROS SERV.DE TERCEIROS PJ- PAGTO ANTECIPADO']
df_cnpj = df[df['CREDOR'] != '160129']

df_cnpj['CREDOR'].unique()

array(['01590728000264', '05462543000144', '08792763000124',
       '11142525000188', '28811718000187', '31053239000153',
       '33658130000175', '37583132000101', '01590728000850',
       '05872523000141', '06981180000116', '09583781000169',
       '12742245000173', '17281106000103', '20584944000160',
       '27325275000151', '27995686000154', '34028316001509',
       '39319890000134', '41505690000118', '41776201000162',
       '43003732000148', '43433811000199', '44812602000119',
       '45424853000199', '76535764000143', '00139736000145',
       '01255291000121', '01804159000121', '02338743000100',
       '02347121000210', '03157268000120', '03539933000140',
       '04785664000164', '05088041000103', '06022475000165',
       '06131805000150', '06894510000136', '07027329743',
       '07457410000296', '07912053000128', '08605233000120',
       '08703140000138', '09035549000196', '09249668000141',
       '09505477000101', '10463704000154', '10526703000101',
       '10842829000195', '1

In [24]:
df_cnpj[df_cnpj['CREDOR'] == '160129']

,UG,SIGLA UG,ACAO,NOME_ACAO,ANO,CREDOR,NOME_CREDOR,DATA,DIAS,ICFEX,NOME_ICFEX,ND,NOME_ND,NDSI,NOME_NDSI,NE,PI,NOME_PI,PROGRAMA,PTRES,TIPO,TIPO_NE,LOCAL,UF_CREDOR,UF_UG,UGR,NOME_UGR,RADPI4,MEIPI3,SUFPI4,A_LIQUIDAR,LIQUIDADO_A_PAGAR,TOTAL_A_PAGAR,PAGO


In [25]:
ano_atual = str(datetime.now().year)


df_corrente = df[df['ANO'] == ano_atual]
df_corrente

,UG,SIGLA UG,ACAO,NOME_ACAO,ANO,CREDOR,NOME_CREDOR,DATA,DIAS,ICFEX,NOME_ICFEX,ND,NOME_ND,NDSI,NOME_NDSI,NE,PI,NOME_PI,PROGRAMA,PTRES,TIPO,TIPO_NE,LOCAL,UF_CREDOR,UF_UG,UGR,NOME_UGR,RADPI4,MEIPI3,SUFPI4,A_LIQUIDAR,LIQUIDADO_A_PAGAR,TOTAL_A_PAGAR,PAGO
0,160129,ESA,156M,MODERNIZACAO E TRANSFORMACAO ESTRATEG,2023,01590728000264,MICROTECNICA INFORMATICA LTDA,2023-11-01,15,160533,4º CGCFEX,449052,EQUIPAMENTOS E MATERIAL PERMANENTE,44905230,MAQUINAS E EQUIPAMENTOS ENERGETICOS,2023NE001319,F8MODCXDEGE,ATENDER NEC DECEX MODERNIZACAO OPERACIONAL,6012,171418,ANO,ORDINARIO,ESCOLA DE SARGENTOS DAS ARMAS,DF,MG,160503,DECEX/GESTOR,F8MO,DCX,DEGE,3720.60,0.0,3720.60,0.0
1,160129,ESA,156M,MODERNIZACAO E TRANSFORMACAO ESTRATEG,2023,05462543000144,EASYTECH INFORMATICA E SERVICOS LTDA,2023-10-31,16,160533,4º CGCFEX,449052,EQUIPAMENTOS E MATERIAL PERMANENTE,44905237,EQUIPAMENTOS DE TIC - ATIVOS DE REDE,2023NE001318,F8MODCXDEGE,ATENDER NEC DECEX MODERNIZACAO OPERACIONAL,6012,171418,ANO,ORDINARIO,ESCOLA DE SARGENTOS DAS ARMAS - AVENIDA SETE ...,DF,MG,160503,DECEX/GESTOR,F8MO,DCX,DEGE,8070.92,0.0,8070.92,0.0
2,160129,ESA,156M,MODERNIZACAO E TRANSFORMACAO ESTRATEG,2023,08792763000124,MEGA BYTE MAGAZINE LTDA,2023-10-31,16,160533,4º CGCFEX,449052,EQUIPAMENTOS E MATERIAL PERMANENTE,44905241,EQUIPAMENTOS DE TIC - COMPUTADORES,2023NE001317,F8MODCXDEGE,ATENDER NEC DECEX MODERNIZACAO OPERACIONAL,6012,171418,ANO,ORDINARIO,ESCOLA DE SARGENTOS DAS ARMAS - AVENIDA SETE ...,SP,MG,160503,DECEX/GESTOR,F8MO,DCX,DEGE,26270.00,0.0,26270.00,0.0
3,160129,ESA,156M,MODERNIZACAO E TRANSFORMACAO ESTRATEG,2023,11142525000188,CLEBER NASCIMENTO DA ROSA,2023-11-01,15,160533,4º CGCFEX,449052,EQUIPAMENTOS E MATERIAL PERMANENTE,44905233,"EQUIPAMENTOS PARA AUDIO, VIDEO E FOTO",2023NE001322,F8MODCXDEGE,ATENDER NEC DECEX MODERNIZACAO OPERACIONAL,6012,171418,ANO,ORDINARIO,ESCOLA DE SARGENTOS DAS ARMAS,SC,MG,160503,DECEX/GESTOR,F8MO,DCX,DEGE,260.00,0.0,260.00,0.0
4,160129,ESA,156M,MODERNIZACAO E TRANSFORMACAO ESTRATEG,2023,28811718000187,BRUNO DIGITAL COMERCIO DE MERCADORIA EM GERAL ...,2023-10-31,16,160533,4º CGCFEX,449052,EQUIPAMENTOS E MATERIAL PERMANENTE,44905241,EQUIPAMENTOS DE TIC - COMPUTADORES,2023NE001316,F8MODCXDEGE,ATENDER NEC DECEX MODERNIZACAO OPERACIONAL,6012,171418,ANO,ORDINARIO,ESCOLA DE SARGENTOS DAS ARMAS - AVENIDA SETE ...,SP,MG,160503,DECEX/GESTOR,F8MO,DCX,DEGE,79002.00,0.0,79002.00,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3333,167129,ESA,2919,REGISTRO E FISCALIZACAO DE PRODUTOS C,2023,65095531000150,LOGOS PAPELARIA LTDA,2023-05-25,175,160533,4º CGCFEX,449052,EQUIPAMENTOS E MATERIAL PERMANENTE,44905245,EQUIPAMENTOS DE TIC - IMPRESSORAS,2023NE001044,E3PCFSCINFO,INFORMATICA,6012,215839,ANO,ORDINARIO,ESCOLA DE SARGENTOS DAS ARMAS - AVENIDA SETE ...,MG,MG,167504,COEX - GESTOR,E3PC,FSC,INFO,0.00,0.0,0.00,1800.0
3334,167129,ESA,2919,REGISTRO E FISCALIZACAO DE PRODUTOS C,2023,81243735000903,POSITIVO TECNOLOGIA S.A.,2023-02-15,274,160533,4º CGCFEX,449052,EQUIPAMENTOS E MATERIAL PERMANENTE,44905241,EQUIPAMENTOS DE TIC - COMPUTADORES,2023NE000261,E3PCFSCINFO,INFORMATICA,6012,215839,ANO,ORDINARIO,ESCOLA DE SARGENTOS DAS ARMAS - AVENIDA SETE ...,BA,MG,167504,COEX - GESTOR,E3PC,FSC,INFO,0.00,0.0,0.00,7114.0
3335,167129,ESA,2919,REGISTRO E FISCALIZACAO DE PRODUTOS C,2023,98488040644,MARIO HENRIQUE CUBBI,2023-05-22,178,160533,4º CGCFEX,339030,MATERIAL DE CONSUMO,33903001,COMBUSTIVEIS E LUBRIFICANTES AUTOMOTIVOS,2023NE001035,E3PCFSCCOLU,COMBUSTIVEIS E LUBRIFICANTES,6012,215839,ANO,ORDINARIO,ESCOLA DE SARGENTOS DAS ARMAS - AVENIDA SETE ...,MG,MG,167504,COEX - GESTOR,E3PC,FSC,COLU,0.00,0.0,0.00,2236.2
3336,167129,ESA,2919,REGISTRO E FISCALIZACAO DE PRODUTOS C,2023,98488040644,MARIO HENRIQUE CUBBI,2023-08-31,77,160533,4º CGCFEX,339030,MATERIAL DE CONSUMO,33903096,MATERIAL DE CONSUMO - PAGTO ANTECIPADO,2023NE001506,E3PCOPFCOLU,COMBUSTIVEL E LUBRIFICANTES,6012,215839,ANO,ORDINARIO,ESCOL

In [26]:
df['ANO'].unique()

array(['2023', '2022', '2021'], dtype=object)

# Pesquisas no DataFrame

In [27]:
# Criando um DataFrame para testes 
df_teste = df.copy()

## Empenhos com mais de 100 dias

### Empenhos da ND 30

In [28]:
# Empenhos da ND 30 com prazo de entrega superior a 100 dias

df_nd30_100dias = df_teste[(df_teste['A_LIQUIDAR'] != 0) & (df_teste['ND'] == '339030') & (df_teste['DIAS'] > 100) & (df_teste['TIPO_NE'] == 'ORDINARIO') ].sort_values(by='DIAS', ascending=False).reset_index(drop=True)

df_nd30_100dias.sort_values(by='A_LIQUIDAR', ascending=False).reset_index(drop=True)

#df_nd30_100dias.to_excel('nd30_PA.xlsx')

,UG,SIGLA UG,ACAO,NOME_ACAO,ANO,CREDOR,NOME_CREDOR,DATA,DIAS,ICFEX,NOME_ICFEX,ND,NOME_ND,NDSI,NOME_NDSI,NE,PI,NOME_PI,PROGRAMA,PTRES,TIPO,TIPO_NE,LOCAL,UF_CREDOR,UF_UG,UGR,NOME_UGR,RADPI4,MEIPI3,SUFPI4,A_LIQUIDAR,LIQUIDADO_A_PAGAR,TOTAL_A_PAGAR,PAGO
0,160129,ESA,8965,CAPACITACAO PROFISSIONAL MILITAR DO E,2023,33345691000114,CONCRETA BRASIL SOLUCOES EMPRESARIAIS LTDA,2023-07-25,114,160533,4º CGCFEX,339030,MATERIAL DE CONSUMO,33903024,MATERIAL P/ MANUT.DE BENS IMOVEIS/INSTALACOES,2023NE000890,C4ENMILCAPE,CAPACITACAO DE PESSOAL,6012,171423,ANO,ORDINARIO,ESCOLA DE SARGENTOS DAS ARMAS,RJ,MG,160503,DECEX/GESTOR,C4EN,MIL,CAPE,11469.19,0.0,11469.19,0.0
1,167129,ESA,2919,REGISTRO E FISCALIZACAO DE PRODUTOS C,2023,13754000000129,"G4 SERVICO E COMERCIO DE PRODUTOS INDUSTRIAIS,...",2023-06-02,167,160533,4º CGCFEX,339030,MATERIAL DE CONSUMO,33903039,MATERIAL P/ MANUTENCAO DE VEICULOS,2023NE001125,E3PCFSCMABM,MANUTENCAO DE BENS MOVEIS,6012,215839,ANO,ORDINARIO,ESCOLA DE SARGENTOS DAS ARMAS,RJ,MG,167504,COEX - GESTOR,E3PC,FSC,MABM,9999.67,0.0,9999.67,0.0
2,160129,ESA,8965,CAPACITACAO PROFISSIONAL MILITAR DO E,2023,20240470000130,R.A.C. CUNHA MATERIAIS P/ CONSTRUCAO LTDA,2023-07-25,114,160533,4º CGCFEX,339030,MATERIAL DE CONSUMO,33903024,MATERIAL P/ MANUT.DE BENS IMOVEIS/INSTALACOES,2023NE000894,C4ENMILCAPE,CAPACITACAO DE PESSOAL,6012,171423,ANO,ORDINARIO,ESCOLA DE SARGENTOS DAS ARMAS,SP,MG,160503,DECEX/GESTOR,C4EN,MIL,CAPE,8846.35,0.0,8846.35,0.0
3,160129,ESA,212B,BENEFICIOS OBRIGATORIOS AOS SERVIDORE,2023,28259514000185,CLEVER FERREIRA COSTA 01175363693,2023-05-03,197,160533,4º CGCFEX,339030,MATERIAL DE CONSUMO,33903021,MATERIAL DE COPA E COZINHA,2023NE000465,E6SUPLJA5PA,A5PA - PASA,0032,171397,ANO,ORDINARIO,ESCOLA DE SARGENTOS DAS ARMAS,MG,MG,160504,COEX/GESTOR,E6SU,PLJ,A5PA,3044.75,0.0,3044.75,0.0
4,160129,ESA,8965,CAPACITACAO PROFISSIONAL MILITAR DO E,2023,39242061000109,CENTRAL TINTAS CURVELO LTDA,2023-07-25,114,160533,4º CGCFEX,339030,MATERIAL DE CONSUMO,33903024,MATERIAL P/ MANUT.DE BENS IMOVEIS/INSTALACOES,2023NE000891,C4ENMILCAPE,CAPACITACAO DE PESSOAL,6012,171423,ANO,ORDINARIO,ESCOLA DE SARGENTOS DAS ARMAS,MG,MG,160503,DECEX/GESTOR,C4EN,MIL,CAPE,552.90,0.0,552.90,0.0
5,160129,ESA,2000,ADMINISTRACAO DA UNIDADE,2023,08605233000120,BALCAO DOS PARAFUSOS E FERRAMENTAS LTDA,2023-07-31,108,160533,4º CGCFEX,339030,MATERIAL DE CONSUMO,33903016,MATERIAL DE EXPEDIENTE,2023NE000940,I3DAFUNADOM,ADMINISTRACAO DE OM,0032,171460,ANO,ORDINARIO,ESCOLA DE SARGENTOS DAS ARMAS - AVENIDA SETE ...,MG,MG,160073,DGO,I3DA,FUN,ADOM,69.00,0.0,69.00,0.0
6,160129,ESA,2000,ADMINISTRACAO DA UNIDADE,2023,08605233000120,BALCAO DOS PARAFUSOS E FERRAMENTAS LTDA,2023-08-01,107,160533,4º CGCFEX,339030,MATERIAL DE CONSUMO,33903025,MATERIAL P/ MANUTENCAO DE BENS MOVEIS,2023NE000948,I3DAFUNADOM,ADMINISTRACAO DE OM,0032,171460,ANO,ORDINARIO,ESCOLA DE SARGENTOS DAS ARMAS - AVENIDA SETE ...,MG,MG,160073,DGO,I3DA,FUN,ADOM,18.70,0.0,18.70,0.0
7,167129,ESA,2919,REGISTRO E FISCALIZACAO DE PRODUTOS C,2023,02852112698,LUCIANO ALVES DE PAULA,2023-06-13,156,160533,4º CGCFEX,339030,MATERIAL DE CONSUMO,33903096,MATERIAL DE CONSUMO - PAGTO ANTECIPADO,2023NE001191,E3PCOPFCOLU,COMBUSTIVEL E LUBRIFICANTES,6012,215839,ANO,ORDINARIO,ESCOLA DE SARGENTOS DAS ARMAS - AVENIDA SETE ...,MS,MG,167504,COEX - GESTOR,E3PC,OPF,COLU,5.22,0.0,5.22,0.0


### Empenhos ND 52

In [29]:
df_nd52_100dias = df_teste[(df_teste['A_LIQUIDAR'] != 0) & (df_teste['ND'] == '449052') & (df_teste['DIAS'] > 50) ].sort_values(by='DIAS', ascending=False).reset_index(drop=True)

df_nd52_100dias.reset_index(drop=True)

#df_nd52_100dias.to_excel('nd52_PA.xlsx')

,UG,SIGLA UG,ACAO,NOME_ACAO,ANO,CREDOR,NOME_CREDOR,DATA,DIAS,ICFEX,NOME_ICFEX,ND,NOME_ND,NDSI,NOME_NDSI,NE,PI,NOME_PI,PROGRAMA,PTRES,TIPO,TIPO_NE,LOCAL,UF_CREDOR,UF_UG,UGR,NOME_UGR,RADPI4,MEIPI3,SUFPI4,A_LIQUIDAR,LIQUIDADO_A_PAGAR,TOTAL_A_PAGAR,PAGO
0,160129,ESA,8965,CAPACITACAO PROFISSIONAL MILITAR DO E,2023,28320324000126,FREDERICO DELGADO DE ALMEIDA,2023-06-19,150,160533,4º CGCFEX,449052,EQUIPAMENTOS E MATERIAL PERMANENTE,44905224,"EQUIPAMENTO DE PROTECAO, SEGURANCA E SOCORRO",2023NE000672,C4ENMILCAPE,CAPACITACAO DE PESSOAL,6012,171423,ANO,ORDINARIO,ESCOLA DE SARGENTOS DAS ARMAS - AVENIDA SETE ...,MG,MG,160503,DECEX/GESTOR,C4EN,MIL,CAPE,1379.10,0.0,1379.10,0.0
1,160129,ESA,8965,CAPACITACAO PROFISSIONAL MILITAR DO E,2023,81243735000903,POSITIVO TECNOLOGIA S.A.,2023-07-24,115,160533,4º CGCFEX,449052,EQUIPAMENTOS E MATERIAL PERMANENTE,44905241,EQUIPAMENTOS DE TIC - COMPUTADORES,2023NE000889,C4ENMILCAPE,CAPACITACAO DE PESSOAL,6012,171423,ANO,ORDINARIO,ESCOLA DE SARGENTOS DAS ARMAS,BA,MG,160503,DECEX/GESTOR,C4EN,MIL,CAPE,11454.00,0.0,11454.00,0.0
2,167129,ESA,2004,ASSISTENCIA MEDICA E ODONTOLOGICA AOS,2023,10902067000175,ESCRIBLU COMERCIO DE MOVEIS LTDA,2023-08-02,106,160533,4º CGCFEX,449052,EQUIPAMENTOS E MATERIAL PERMANENTE,44905242,MOBILIARIO EM GERAL,2023NE001377,D5SAFUSASOC,ASSISTENCIA SOCIAL,0032,215846,ANO,ORDINARIO,ESCOLA DE SARGENTOS DAS ARMAS,SC,MG,167505,DGP,D5SA,FUS,ASOC,2707.96,0.0,2707.96,0.0
3,167129,ESA,2004,ASSISTENCIA MEDICA E ODONTOLOGICA AOS,2023,10902067000175,ESCRIBLU COMERCIO DE MOVEIS LTDA,2023-08-02,106,160533,4º CGCFEX,449052,EQUIPAMENTOS E MATERIAL PERMANENTE,44905242,MOBILIARIO EM GERAL,2023NE001378,D5SAFUSASOC,ASSISTENCIA SOCIAL,0032,215846,ANO,ORDINARIO,ESCOLA DE SARGENTOS DAS ARMAS,SC,MG,167505,DGP,D5SA,FUS,ASOC,3047.67,0.0,3047.67,0.0
4,167129,ESA,2004,ASSISTENCIA MEDICA E ODONTOLOGICA AOS,2023,45169346000156,FRANCILANDIO THIERES DE CARVALHO SILVA,2023-08-10,98,160533,4º CGCFEX,449052,EQUIPAMENTOS E MATERIAL PERMANENTE,44905212,APARELHOS E UTENSILIOS DOMESTICOS,2023NE001405,D5SAFUSASOC,ASSISTENCIA SOCIAL,0032,215846,ANO,ORDINARIO,ESCOLA DE SARGENTOS DAS ARMAS - AVENIDA SETE ...,PI,MG,167505,DGP,D5SA,FUS,ASOC,3964.69,0.0,3964.69,0.0
5,167129,ESA,2004,ASSISTENCIA MEDICA E ODONTOLOGICA AOS,2023,48489837000172,JB LICITACOES LTDA,2023-08-10,98,160533,4º CGCFEX,449052,EQUIPAMENTOS E MATERIAL PERMANENTE,44905242,MOBILIARIO EM GERAL,2023NE001408,D5SAFUSASOC,ASSISTENCIA SOCIAL,0032,215846,ANO,ORDINARIO,ESCOLA DE SARGENTOS DAS ARMAS - AVENIDA SETE ...,SC,MG,167505,DGP,D5SA,FUS,ASOC,2460.00,0.0,2460.00,0.0
6,167129,ESA,2000,ADMINISTRACAO DA UNIDADE,2023,81243735000903,POSITIVO TECNOLOGIA S.A.,2023-08-18,90,160533,4º CGCFEX,449052,EQUIPAMENTOS E MATERIAL PERMANENTE,44905241,EQUIPAMENTOS DE TIC - COMPUTADORES,2023NE001414,C1ENCONDETM,CONCURSOS AS ESCOLAS MILITARES,0032,171502,ANO,ORDINARIO,ESCOLA DE SARGENTOS DAS ARMAS - AVENIDA SETE ...,BA,MG,167503,DECEX - GESTOR,C1EN,CON,DETM,21864.00,0.0,21864.00,0.0


### Consolidação ND 30 e ND 52

In [30]:
nd = ['449052', '339030' ]

cols = ['UG', 'ANO', 'CREDOR', 'NOME_CREDOR',
       'DATA', 'DIAS', 'ND', 'NDSI', 'NOME_NDSI', 
       'NE', 'PI', 'NOME_PI', 'UGR', 'NOME_UGR', 
       'A_LIQUIDAR', 'LIQUIDADO_A_PAGAR','PAGO']

df_foco = df_teste.loc[df_teste['ND'].isin(nd) & (df_teste['DIAS'] > 60) & (df_teste['A_LIQUIDAR'] != 0), :].sort_values(by='DIAS', ascending=False).reset_index(drop=True)

df_foco.reset_index(drop=True)

df_foco = df_foco.loc[:, cols]

# Baixando foco para Almox
df_foco.to_excel('Retorno_Almox.xlsx')

### Recursos Liquidados a pagar

In [31]:
df_liqui_a_pagar =df_teste[ df_teste['LIQUIDADO_A_PAGAR'] != 0 ]

df_liqui_a_pagar.sort_values(by='LIQUIDADO_A_PAGAR', ascending=False).reset_index(drop=True).head(2)

,UG,SIGLA UG,ACAO,NOME_ACAO,ANO,CREDOR,NOME_CREDOR,DATA,DIAS,ICFEX,NOME_ICFEX,ND,NOME_ND,NDSI,NOME_NDSI,NE,PI,NOME_PI,PROGRAMA,PTRES,TIPO,TIPO_NE,LOCAL,UF_CREDOR,UF_UG,UGR,NOME_UGR,RADPI4,MEIPI3,SUFPI4,A_LIQUIDAR,LIQUIDADO_A_PAGAR,TOTAL_A_PAGAR,PAGO
0,167129,ESA,2004,ASSISTENCIA MEDICA E ODONTOLOGICA AOS,2023,17845504000296,UNIMED VARGINHA COOPERATIVA DE TRABALHO MEDICO,2023-11-01,15,160533,4º CGCFEX,339039,OUTROS SERVICOS DE TERCEIROS - PESSOA JURIDICA,33903950,"SERV.MEDICO-HOSPITAL.,ODONTOL.E LABORATORIAIS",2023NE001745,D8SAFUSOCSA,ORGANIZACAO CIVIL DE SAUDE,0032,215845,ANO,ESTIMATIVO,CLINICA/CONSULTÓRIO/HOSPITAL,MG,MG,167505,DGP,D8SA,FUS,OCSA,0.0,26215.31,26215.31,0.0
1,160129,ESA,21A0,APRESTAMENTO DAS FORCAS,2023,38073038000167,WR NUTRICAO ANIMAL LTDA,2023-09-01,76,160533,4º CGCFEX,339030,MATERIAL DE CONSUMO,33903006,ALIMENTOS PARA ANIMAIS,2023NE001097,E6RVPLJALIE,ALIMENTACAO PARA EQUINOS.,6012,193910,ANO,GLOBAL,ESCOLA DE SARGENTOS DAS ARMAS - AVENIDA SETE ...,ES,MG,160504,COEX/GESTOR,E6RV,PLJ,ALIE,39408.2,18400.00,57808.20,0.0


In [32]:
total_liqui_a_pagar = df_liqui_a_pagar['LIQUIDADO_A_PAGAR'].sum()
locale.currency(total_liqui_a_pagar, grouping=True)

'R$ 119.130,13'

In [33]:
df_aux = df_liqui_a_pagar.groupby(by='NOME_NDSI')['LIQUIDADO_A_PAGAR'].sum().sort_values( ascending=False).reset_index()

fig = px.bar(df_aux,
             x = 'NOME_NDSI',
             y = 'LIQUIDADO_A_PAGAR',
             log_y=True,
             height=900,
             title='Saldos liquidados a pagar por NDSI',
             labels={'NOME_NDSI':'Descrição do Subitem', 'LIQUIDADO_A_PAGAR':'Liquidado a Pagar'},
             text_auto='.3s',
             color='LIQUIDADO_A_PAGAR',
             color_continuous_scale = 'reds',
             template='plotly_dark')
fig.update_traces(textposition = 'outside', cliponaxis = False, hovertemplate=None)
fig.update_layout(hovermode="x unified")

fig.show()

In [34]:
df_teste[df_teste['NE'] == '2023NE000392']

,UG,SIGLA UG,ACAO,NOME_ACAO,ANO,CREDOR,NOME_CREDOR,DATA,DIAS,ICFEX,NOME_ICFEX,ND,NOME_ND,NDSI,NOME_NDSI,NE,PI,NOME_PI,PROGRAMA,PTRES,TIPO,TIPO_NE,LOCAL,UF_CREDOR,UF_UG,UGR,NOME_UGR,RADPI4,MEIPI3,SUFPI4,A_LIQUIDAR,LIQUIDADO_A_PAGAR,TOTAL_A_PAGAR,PAGO
362,160129,ESA,212B,BENEFICIOS OBRIGATORIOS AOS SERVIDORE,2023,31669124000198,LS REFRIGERACAO LTDA,2023-04-18,212,160533,4º CGCFEX,449052,EQUIPAMENTOS E MATERIAL PERMANENTE,44905228,MAQUINAS E EQUIPAMENTOS DE NATUREZA INDUSTRIAL,2023NE000392,E6SUPLJA5PA,A5PA - PASA,0032,171397,ANO,ORDINARIO,"ESCOLA DE SARGENTOS DAS ARMAS, AVENIDA SETE DE...",SC,MG,160504,COEX/GESTOR,E6SU,PLJ,A5PA,0.0,0.0,0.0,8837.42
3118,167129,ESA,2004,ASSISTENCIA MEDICA E ODONTOLOGICA AOS,2023,46940235787,DAVILAN DE SOUZA PORCARO,2023-02-27,262,160533,4º CGCFEX,339036,OUTROS SERVICOS DE TERCEIROS - PESSOA FISICA,33903630,SERVICOS MEDICOS E ODONTOLOGICOS,2023NE000392,D8SAFUSPRSA,PROFISSIONAL DE SAUDE AUTONOMO,0032,215845,ANO,ESTIMATIVO,CLINICA/CONSULTÓRIO/HOSPITAL,MG,MG,167505,DGP,D8SA,FUS,PRSA,0.0,0.0,0.0,648.27


In [37]:
df_restos = df_teste[(df_teste['ANO'] == '2022') & (df_teste['A_LIQUIDAR'] !=  0)].sort_values(by='A_LIQUIDAR', ascending=False)

# f_restos.to_excel('Restos_a_pagar.xlsx')